# TP3 – Régression Linéaire Multivariable sur données biologiques
# Objectif du TP

"""
L'objectif de ce TP est de réaliser un projet complet de régression linéaire multivariable
pour prédire des valeurs continues à partir de données d'expression génique.

Dans ce TP, nous allons :
- Utiliser la régression linéaire pour prédire une variable continue
- Implémenter manuellement l'algorithme sans utiliser de bibliothèques ML
- Calculer les coefficients par la méthode des moindres carrés
- Évaluer les performances avec R², MSE, RMSE, MAE
"""

# ==============================================================================
# PARTIE 1 : PRÉTRAITEMENT DES DONNÉES
# ==============================================================================

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Création du répertoire de travail
BASE_DIR = "/home/youcef/bioinfo/fd"
os.makedirs(BASE_DIR, exist_ok=True)

# Chargement du dataset
file_path = "/home/youcef/bioinfo/fd/AirQualityUCI.csv"

if not os.path.exists(file_path):
    print(f"Le fichier n'existe pas : {file_path}")
    print("Veuillez vérifier le chemin d'accès.")
else:
    df = pd.read_csv(file_path)
    
    print("\nAperçu du jeu de données original :")
    print(df.head())
    print(f"\nDimensions : {df.shape[0]} lignes × {df.shape[1]} colonnes")
    
    # Vérification des valeurs manquantes
    missing_counts = df.isnull().sum()
    total_missing = missing_counts.sum()
    
    if total_missing > 0:
        print(f"\nValeurs manquantes détectées : {total_missing}")
        # Remplacement par la moyenne
        for col in df.select_dtypes(include=[np.number]).columns:
            if df[col].isnull().sum() > 0:
                df[col].fillna(df[col].mean(), inplace=True)
        print("Valeurs manquantes remplacées par la moyenne")
    else:
        print("\nAucune valeur manquante détectée")
    
    # Suppression des doublons
    nb_doublons = df.duplicated().sum()
    if nb_doublons > 0:
        print(f"\n⚠️ {nb_doublons} doublon(s) détecté(s)")
        df.drop_duplicates(inplace=True)
        print("Doublons supprimés")
    else:
        print("Aucun doublon détecté")
    
    # Pour la régression, nous allons créer une variable cible continue
    # Nous allons utiliser la moyenne d'expression des premiers gènes comme variable à prédire
    print("\n🎯 Création de la variable cible pour la régression...")
    
    # Sélection des 10 premiers gènes pour créer notre variable cible
    target_genes = df.columns[0:10].tolist()
    df['target_expression'] = df[target_genes].mean(axis=1)
    
    print(f"Variable cible créée : 'target_expression'")
    print(f"Statistiques de la variable cible :")
    print(df['target_expression'].describe())
    
    # Normalisation (standardisation Z-score pour la régression)
    df_norm = df.copy()
    
    # On exclut la variable cible et sample_type_id de la normalisation des features
    features_to_normalize = [col for col in df.columns 
                            if col not in ['target_expression', 'sample_type_id'] + target_genes]
    
    for col in features_to_normalize:
        mean_val = df[col].mean()
        std_val = df[col].std()
        if std_val != 0:
            df_norm[col] = (df[col] - mean_val) / std_val
        else:
            df_norm[col] = 0.0
    
    print("\n✅ Standardisation Z-score effectuée sur les features")
    print("\n📊 Aperçu des données normalisées :")
    print(df_norm.head())
    
    # Sauvegarde
    output_preprocessed = f"{BASE_DIR}/Liver_RNA_preprocessed_LR.csv"
    df_norm.to_csv(output_preprocessed, index=False)
    print(f"\n💾 Données prétraitées sauvegardées : {output_preprocessed}")


Aperçu du jeu de données original :
                                                                                                            Date;Time;CO(GT);PT08.S1(CO);NMHC(GT);C6H6(GT);PT08.S2(NMHC);NOx(GT);PT08.S3(NOx);NO2(GT);PT08.S4(NO2);PT08.S5(O3);T;RH;AH;;
10/03/2004;18.00.00;2            6;1360;150;11                 9;1046;166;1056;113;1692;1268;13 6;48 9;0                                                7578;;                                                                          
10/03/2004;19.00.00;2;1292;112;9 4;955;103;1174;92;1559;972;13 3;47                             7;0  7255;;                                                NaN                                                                          
10/03/2004;20.00.00;2            2;1402;88;9                   0;939;131;1140;114;1555;1074;11  9;54 0;0                                                7502;;                                                                          
10/03/2004;21.00.00;2          

TypeError: Could not convert ['7578;;' 0 '7502;;' ... '7531;;' '7119;;' '5028;;'] to numeric

# ==============================================================================
# PARTIE 2 : DIVISION DU JEU DE DONNÉES (80% TRAIN / 20% TEST)
# ==============================================================================

In [ ]:
from sklearn.model_selection import train_test_split

# Préparation des données : on exclut les gènes utilisés pour créer la cible
features_columns = [
    col
    for col in df_norm.columns
    if col not in ["target_expression", "sample_type_id"] + target_genes
]

X = df_norm[features_columns]
y = df_norm["target_expression"]

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"\n✅ Taille du train set : X={X_train.shape}, y={y_train.shape}")
print(f"✅ Taille du test set : X={X_test.shape}, y={y_test.shape}")

# Sauvegarde des fichiers
train_df = X_train.copy()
train_df["target_expression"] = y_train

test_df = X_test.copy()
test_df["target_expression"] = y_test

train_path = f"{BASE_DIR}/Liver_RNA_train_LR.csv"
test_path = f"{BASE_DIR}/Liver_RNA_test_LR.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"\n💾 Fichier d'entraînement : {train_path}")
print(f"💾 Fichier de test : {test_path}")

# ==============================================================================
# PARTIE 3 : IMPLÉMENTATION MANUELLE DE LA RÉGRESSION LINÉAIRE MULTIVARIABLE
# ==============================================================================

In [ ]:
class MultipleLinearRegression:
    """
    Implémentation manuelle de la régression linéaire multivariable.

    Modèle : y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ

    Méthode des moindres carrés :
    β = (X^T X)^(-1) X^T y

    où X est la matrice des features avec une colonne de 1 pour l'intercept
    """

    def __init__(self):
        self.coefficients = None
        self.intercept = None
        self.n_features = None

    def fit(self, X, y):
        """
        Entraînement du modèle par la méthode des moindres carrés.

        Args:
            X : matrice des features (n_samples, n_features)
            y : vecteur cible (n_samples,)
        """
        # Conversion en arrays numpy
        X = np.array(X)
        y = np.array(y)

        self.n_features = X.shape[1]

        # Ajout d'une colonne de 1 pour l'intercept
        X_with_intercept = np.column_stack([np.ones(len(X)), X])

        # Calcul des coefficients : β = (X^T X)^(-1) X^T y
        # Méthode 1 : Inversion de matrice (peut être instable)
        # XtX = X_with_intercept.T @ X_with_intercept
        # Xty = X_with_intercept.T @ y
        # beta = np.linalg.inv(XtX) @ Xty

        # Méthode 2 : Résolution directe du système (plus stable)
        XtX = X_with_intercept.T @ X_with_intercept
        Xty = X_with_intercept.T @ y

        # Ajout d'un terme de régularisation pour stabilité numérique
        ridge_penalty = 1e-6
        XtX_reg = XtX + ridge_penalty * np.eye(XtX.shape[0])

        beta = np.linalg.solve(XtX_reg, Xty)

        # Séparation intercept et coefficients
        self.intercept = beta[0]
        self.coefficients = beta[1:]

        print(f"\n✅ Modèle entraîné avec succès !")
        print(f"   Intercept (β₀) : {self.intercept:.6f}")
        print(f"   Nombre de coefficients : {len(self.coefficients)}")
        print(f"   Exemple de coefficients : {self.coefficients[:5]}")

    def predict(self, X):
        """
        Prédiction des valeurs cibles.

        Args:
            X : matrice des features

        Returns:
            Prédictions (array)
        """
        X = np.array(X)
        return self.intercept + X @ self.coefficients

    def get_coefficients(self):
        """Retourne les coefficients du modèle"""
        return {"intercept": self.intercept, "coefficients": self.coefficients}


# Entraînement du modèle
print("\n🔧 Entraînement du modèle de régression linéaire multivariable...")
lr_model = MultipleLinearRegression()
lr_model.fit(X_train.values, y_train.values)

# ==============================================================================
# PARTIE 4 : ÉVALUATION SUR LE JEU D'ENTRAÎNEMENT
# ==============================================================================

In [ ]:
def calculate_regression_metrics(y_true, y_pred):
    """
    Calcul manuel des métriques de régression.

    Métriques calculées :
    - MSE (Mean Squared Error) : moyenne des erreurs au carré
    - RMSE (Root Mean Squared Error) : racine du MSE
    - MAE (Mean Absolute Error) : moyenne des erreurs absolues
    - R² (Coefficient de détermination) : proportion de variance expliquée
    """
    n = len(y_true)

    # Erreurs
    errors = y_true - y_pred
    squared_errors = errors**2
    absolute_errors = np.abs(errors)

    # MSE
    mse = np.mean(squared_errors)

    # RMSE
    rmse = np.sqrt(mse)

    # MAE
    mae = np.mean(absolute_errors)

    # R² = 1 - (SS_res / SS_tot)
    ss_res = np.sum(squared_errors)  # Somme des carrés résiduels
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # Somme totale des carrés
    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0

    # R² ajusté (prend en compte le nombre de features)
    # Pas calculé ici car nécessite le nombre de features

    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "n_samples": n}


# Prédictions sur le train set
y_train_pred = lr_model.predict(X_train.values)

# Calcul des métriques
metrics_train = calculate_regression_metrics(y_train.values, y_train_pred)

print("\n📊 RÉSULTATS SUR LE TRAIN SET :")
print(f"  Nombre d'échantillons : {metrics_train['n_samples']}")
print(f"\n  📈 Métriques de performance :")
print(f"  MSE (Mean Squared Error)      : {metrics_train['MSE']:.6f}")
print(f"  RMSE (Root Mean Squared Error): {metrics_train['RMSE']:.6f}")
print(f"  MAE (Mean Absolute Error)     : {metrics_train['MAE']:.6f}")
print(f"  R² (Coefficient de déterminat): {metrics_train['R2']:.6f}")

# Interprétation du R²
if metrics_train["R2"] >= 0.9:
    interpretation = "Excellent (>90% de variance expliquée)"
elif metrics_train["R2"] >= 0.7:
    interpretation = "Bon (70-90% de variance expliquée)"
elif metrics_train["R2"] >= 0.5:
    interpretation = "Modéré (50-70% de variance expliquée)"
else:
    interpretation = "Faible (<50% de variance expliquée)"

print(f"\n  📝 Interprétation du R² : {interpretation}")

# Visualisation : Valeurs réelles vs prédites
plt.figure(figsize=(10, 6))
plt.scatter(y_train.values, y_train_pred, alpha=0.5, s=30)
plt.plot(
    [y_train.min(), y_train.max()],
    [y_train.min(), y_train.max()],
    "r--",
    lw=2,
    label="Prédiction parfaite",
)
plt.xlabel("Valeurs Réelles")
plt.ylabel("Valeurs Prédites")
plt.title(f"Régression Linéaire - Train Set (R² = {metrics_train['R2']:.4f})")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f"{BASE_DIR}/predictions_train_LR.png", dpi=300)
print("\n💾 Graphique prédictions/réalité sauvegardé (train)")

# Distribution des résidus
residuals_train = y_train.values - y_train_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
